In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkFiles
from pyspark.sql import DataFrame
import pyspark.sql.functions as F

In [2]:
# get or create Spark session

app_name = "spark-s3-to-rds"
spark = SparkSession.builder.appName(app_name).getOrCreate()

In [3]:
employee_file ="/FileStore/tables/employee.csv"
employee_file

Out[5]: '/FileStore/tables/employee.csv'

In [4]:
df_employee = spark.read \
    .format("com.databricks.spark.csv") \
    .options(header='true', inferSchema="true") \
    .load(employee_file)
df_employee.printSchema()

root
-- _c0: integer (nullable = true)
-- Employee ID: integer (nullable = true)
-- Email: string (nullable = true)
-- Position: string (nullable = true)
-- DOB: string (nullable = true)
-- Marital Status: string (nullable = true)
-- Gender: string (nullable = true)
-- Hire Date: string (nullable = true)
-- Salaried: integer (nullable = true)
-- Vacation Hours: integer (nullable = true)
-- Sick Leave Hours: integer (nullable = true)
-- Encrypted Password: string (nullable = true)
-- Modified: string (nullable = true)

In [5]:
import boto3

secret_name = "ut/postgres/db"
region_name = "us-east-2"
access_key = "YOUR ACCESS HERE"
secret_key = "YOUR KEY HERE"

session = boto3.session.Session(aws_access_key_id=access_key, aws_secret_access_key=secret_key, region_name=region_name)
client = session.client('secretsmanager')
secret_value = client.get_secret_value(SecretId=secret_name)
# secret_value

In [6]:
import json
def get_connection(secret_value):
  return json.loads(secret_value['SecretString'])
# get_connection(secret_value)

In [7]:
connection = get_connection(secret_value)

# Postgres credentials
jdbcHostname = connection['host']
jdbcPort = connection['port']
jdbcDatabase = "postgres"
dialect = "postgresql"
jdbcUsername = connection['username']
jdbcPassword = connection['password']

jdbcUrl = f"jdbc:{dialect}://{jdbcHostname}:{jdbcPort}/{jdbcDatabase}"
connectionProperties = {
  "user" : jdbcUsername,
  "password" : jdbcPassword,
  "driver" : "org.postgresql.Driver" 
}
# for mysql driver = com.mysql.jdbc.Driver

In [8]:
# jdbcUrl

In [9]:
table = "employee"
mode = "overwrite" # options are: error, append, overwrite

df_employee.write.jdbc(jdbcUrl, table, mode, connectionProperties)